In [46]:
# imports
import os, sys
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

# import tools
#sys.path.append('../../../shared') temp for demo
sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\shared')
sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared')
import arc, satfetcher, tools

# import gdvspectra module
#sys.path.append('../../../modules') temp for demo
sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\modules')
sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules')
import gdvspectra, cog

# notify user
arcpy.AddMessage('Beginning GDVSpectra Likelihood.')

In [47]:
# prepare wet, dry season lists todo put into a func
in_months = '9, 10, 11'
in_months = [int(e) for e in in_months.split(',')]

In [48]:
# load netcdf file as xr
in_raw_nc = r'C:\Users\Lewis\Desktop\working\yandi_13_20.nc'
ds = satfetcher.load_local_nc(nc_path=in_raw_nc, 
                              use_dask=True, 
                              conform_nodata_to=np.nan)

Converting netcdf to an xarray dataset.
Converted netcdf to xarray dataset: C:\Users\Lewis\Desktop\working\yandi_13_20.nc
Netcdf converted to xarray dataset successfully.


In [49]:
ds

<xarray.Dataset>
Dimensions:       (time: 344, x: 104, y: 121)
Coordinates:
  * time          (time) datetime64[ns] 2013-03-29T02:02:38 ... 2020-12-28T02...
  * y             (y) float64 -2.514e+06 -2.514e+06 ... -2.517e+06 -2.517e+06
  * x             (x) float64 -1.288e+06 -1.288e+06 ... -1.285e+06 -1.285e+06
    spatial_ref   int32 3577
Data variables:
    nbart_blue    (time, y, x) float32 dask.array<chunksize=(344, 121, 104), meta=np.ndarray>
    nbart_green   (time, y, x) float32 dask.array<chunksize=(344, 121, 104), meta=np.ndarray>
    nbart_red     (time, y, x) float32 dask.array<chunksize=(344, 121, 104), meta=np.ndarray>
    nbart_nir     (time, y, x) float32 dask.array<chunksize=(344, 121, 104), meta=np.ndarray>
    nbart_swir_1  (time, y, x) float32 dask.array<chunksize=(344, 121, 104), meta=np.ndarray>
    nbart_swir_2  (time, y, x) float32 dask.array<chunksize=(344, 121, 104), meta=np.ndarray>
    oa_fmask      (time, y, x) float32 dask.array<chunksize=(344, 121, 104), meta=np.ndarray>
Attributes:
    crs:               EPSG:3577
    grid_mapping:      spatial_ref
    transform:         [ 3.00000e+01  0.00000e+00 -1.28823e+06  0.00000e+00 -...
    res:               30
    nodatavals:        nan
    orig_collections:  ['ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3']
    orig_bbox:         [119.32132693 -22.77485339 119.34873642 -22.74501035]
    orig_slc_off:      False
    orig_resample:     Nearest

In [7]:
# remove mask
in_max_cloud = 10
in_fmask_flags = [1, 4, 5]
ds = cog.remove_fmask_dates(ds=ds, 
                            valid_class=in_fmask_flags, 
                            max_invalid=in_max_cloud, 
                            mask_band='oa_fmask', 
                            nodata_value=np.nan, 
                            drop_fmask=True)  

Removing dates where too many invalid pixels.
Mask band is currently dask. Computing, please wait.
Filling invalid pixels with nan
Dropping mask band.


In [9]:
# capture attributes
band_attrs = ds['nbart_blue'].attrs

In [10]:
# conform bands
in_platform = 'landsat'
ds = satfetcher.conform_dea_ard_band_names(ds=ds, 
                                           platform=in_platform.lower())

Conforming DEA ARD satellite band names.
Satellite band names conformed successfully.


In [12]:
# subset months
ds = gdvspectra.subset_months(ds=ds, 
                              month=in_months,
                              inplace=True)

Subsetting down to specified months.
Reducing dataset into months: [9, 10, 11].
Subset to requested months successfully.


In [14]:
# calculate index
ds = tools.calculate_indices(ds=ds, 
                             index=['tcg', 'tcb'], 
                             rescale=False, 
                             drop=True)

Calculating indices: ['tcg', 'tcb'].
Calculating index: tcg
Calculating index: tcb
Calculated indices successfully.


In [16]:
# add band attrs back on
ds['tcg'].attrs = band_attrs   
ds['tcb'].attrs = band_attrs  

In [18]:
# compute 
ds = ds.compute()

In [20]:
# reduce all selected months into annual medians (year starts, YS)
ds = gdvspectra.resample_to_freq_medians(ds=ds,
                                         freq='YS',
                                         inplace=True)

Resampling dataset down to annual medians.
Resampled down to annual medians successfully.


In [22]:
# interpolate all missing pixels using full linear interpolation
ds = gdvspectra.interp_empty(ds=ds,
                             method='full',
                             inplace=True)

Interpolating empty values in dataset.
Interpolated empty values successfully.


In [24]:
# standardise to targets
ds = gdvspectra.standardise_to_targets(ds, 
                                       q_upper=0.99, 
                                       q_lower=0.05)

Standardising data using invariant targets.
Calculating invariant target sites.
Got orthogonal polynomial coefficients: [-7. -5. -3. -1.  1.  3.  5.  7.]
Got sum of squares: 168 and constant 2: 
Created invariant target sites successfully.
Standardising to invariant targets, rescaling via fuzzy sigmoidal.
Standardised using invariant targets successfully.


In [44]:
# generate cva
ds_cva = gdvspectra.perform_cva(ds=ds,
                                base_times=(2013, 2013),
                                comp_times=(2016, 2020),
                                reduce_comp=False,
                                vege_var = 'tcg',
                                soil_var = 'tcb',
                                tmf=0)

Performing CVA.
Doing CVA: 1.
Doing CVA: 2.
Doing CVA: 3.
Doing CVA: 4.
Doing CVA: 5.
Performed CVA successfully.


In [45]:
ds_cva['angle'].attrs = band_attrs 
ds_cva['magnitude'].attrs = band_attrs 
ds_cva.to_netcdf(r'C:\Users\Lewis\Desktop\working\cva.nc')